In [1]:
# install requirements
!pip install diffusers einops gradio typing_extensions  fastapi==0.100.0 accelerate

# install ip-adapter
%cd /content
!git clone https://github.com/sagiodev/IP-Adapter-Negative.git

# download the models
%cd IP-Adapter-Negative
%mkdir models
%cd models
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin
%mkdir image_encoder
%cd image_encoder
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/pytorch_model.bin
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/config.json
%cd /content/IP-Adapter-Negative

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 6.6 MB/s 

In [ ]:
# Run Negative IP adapter demo
share_public = True # whether creates a public link

import torch
import gc
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter import IPAdapterPlus, IPAdapter
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "models/image_encoder"
ip_ckpt = "models/ip-adapter-plus_sd15.bin"
device = "cuda"
noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16).to(device)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
).to(device)

gc.collect()
torch.cuda.empty_cache()
import random
ip_adapters_list = ['IP Adapter', 'IP Adapter Plus']
ip_model = None
def generate(ip_adapter, image,
             neg_image,
             scale,
             scale_start,
             scale_stop,
             neg_scale,
             scale_neg_start,
             scale_neg_stop,
             prompt,
             negative_prompt,
             num_samples,
             steps,
             guidance_scale,
             seed):
    global ip_model,pipe
    if image is None and neg_image is None:
      scale =0
      neg_scale = 0
      image = Image.open("assets/images/statue.png")

    if seed == -1:
      seed = random.randint(0, 1e12)

    gc.collect()
    torch.cuda.empty_cache()

    if ip_adapter == 'IP Adapter':
      # load ip-adapter
      ip_ckpt = "models/ip-adapter_sd15.bin"
      if type(ip_model) is not IPAdapter:
        print('Load new IP Adapter')
        del ip_model
        gc.collect()
        torch.cuda.empty_cache()
        ip_model = IPAdapter(pipe, image_encoder_path, ip_ckpt, device)
    elif ip_adapter == 'IP Adapter Plus':
      # load ip-adapter
      if type(ip_model) is not IPAdapterPlus:
        print('Load new IP Adapter Plus')
        del ip_model
        gc.collect()
        torch.cuda.empty_cache()
        ip_ckpt = "models/ip-adapter-plus_sd15.bin"
        ip_model = IPAdapterPlus(pipe, image_encoder_path, ip_ckpt, device, num_tokens=16)
    else:
      raise ValueError('IP adapter %s is undefined.'%ip_adapter)

    gc.collect()
    torch.cuda.empty_cache()

    images = ip_model.generate(pil_image=image,
                           negative_pil_image=neg_image,
                           prompt = prompt,
                           negative_prompt= negative_prompt,
                           num_samples=num_samples,
                           scale=scale,  # weight for image prompt
                           scale_start= scale_start,
                           scale_stop= scale_stop,
                           scale_neg = neg_scale, # weight for negative image prompt
                           scale_neg_start = scale_neg_start,
                           scale_neg_stop = scale_neg_stop,
                           num_inference_steps=steps,
                           guidance_scale = guidance_scale,
                           seed=seed)
    gc.collect()
    torch.cuda.empty_cache()
    return images


shortcut_js = """
<script>
function shortcuts(e) {

    if ((event.keyCode == 10 || event.keyCode == 13) && event.ctrlKey) {
        document.getElementById("generate-button").click();
    }
}
document.addEventListener('keyup', shortcuts, false);
</script>
"""

import gradio as gr
with gr.Blocks(head=shortcut_js) as demo:
  resized_image_file = gr.File(visible=False)

  gr.Markdown("# Stable Diffuson Negative Image Prompt")
  with gr.Row():
    with gr.Column():
      with gr.Row():
        with gr.Column():
          prompt = gr.Textbox(label="Prompt", value = "")
          scale = gr.Slider(0, 2, value=1, label="Image prompt weight")
          scale_start = gr.Slider(0, 1, value=0, label="Starting step")
          scale_stop = gr.Slider(0, 1, value=1, label="Ending step")
          imagePrompt = gr.Image(label="Image Prompt", type = "pil")
        with gr.Column():
          negativePrompt = gr.Textbox(label="Negative Prompt", value = "")
          negativeScale = gr.Slider(0, 2, value=1, label="Negative Image prompt weight")
          negativeScale_start = gr.Slider(0, 1, value=0, label="Starting step")
          negativeScale_stop = gr.Slider(0, 1, value=1, label="Ending step")
          negativeImagePrompt = gr.Image(label="Negative Image Prompt", type = "pil")
      with gr.Accordion(label="Advanced options", open=False):
        ip_adapter = gr.Dropdown(choices = ip_adapters_list, value = ip_adapters_list[0], label = "IP Adapter", interactive = True)
        numImages = gr.Number(precision=0, label="Number of images", value=4)
        steps = gr.Number(precision=0, label="Number of steps", value=25)
        guidance_scale = gr.Number(precision=1, label="CFG Scale", value=7.5)
        seed = gr.Number(precision=0, label="Seed", value=-1)
    with gr.Column():
      btn = gr.Button("Generate", elem_id="generate-button")
      imagesOut = gr.Gallery(object_fit="contain", height="auto", selected_index = 0, preview = True)


  inputs = [ip_adapter, imagePrompt, negativeImagePrompt, scale, scale_start, scale_stop, negativeScale, negativeScale_start, negativeScale_stop, prompt, negativePrompt, numImages, steps, guidance_scale, seed]
  outputs = imagesOut
  btn.click(generate, inputs=inputs, outputs=outputs)
  demo.queue().launch(debug=True, share=share_public, inline=False, show_error=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

Load new IP Adapter
positive and negative image


  0%|          | 0/25 [00:00<?, ?it/s]

positive and negative image


  0%|          | 0/25 [00:00<?, ?it/s]